In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## MLP モデルのPyTorchによる実装
基礎編で使った2次元データを基に、MLPモデルをPyTorchで書いてみます。

In [ ]:

# 二次元ガウス分布と一様分布
def get_dataset_2():
    from numpy.random import default_rng
    rng = default_rng(seed=0)  # 今回はデータセットの乱数を固定させます。

    num_signal = 100  # 生成するシグナルイベントの数
    num_background = 1000  # 生成するバックグラウンドイベントの数

    # データ点の生成
    ## 平均(x1,x2) = (1.0, 0.0)、分散=1の２次元ガウス分布
    x_sig = rng.multivariate_normal(mean=[1.0, 0],
                                    cov=[[1, 0], [0, 1]],
                                    size=num_signal)
    t_sig = np.ones((num_signal, 1))  # Signalは1にラベリング

    ## (-5, +5)の一様分布
    x_bg = rng.uniform(low=-5, high=5, size=(num_background, 2))
    t_bg = np.zeros((num_background, 1))  # Backgroundは0にラベリング

    # 2つのラベルを持つ学習データを1つにまとめる
    x = np.concatenate([x_sig, x_bg])
    t = np.concatenate([t_sig, t_bg])

    # データをランダムに並び替える
    p = rng.permutation(len(x))
    x, t = x[p], t[p]

    return x, t


# ラベル t={0,1}を持つデータ点のプロット
def plot_datapoint(x, t):
    # シグナル/バックグラウンドの抽出
    xS = x[t[:, 0] == 1]  # シグナルのラベルだけを抽出
    xB = x[t[:, 0] == 0]  # バックグラウンドのラベルだけを抽出

    # プロット
    plt.scatter(xS[:, 0], xS[:, 1], label='Signal', c='red', s=10)  # シグナルをプロット
    plt.scatter(xB[:, 0], xB[:, 1], label='Background', c='blue', s=10)  # バックグラウンドをプロット
    plt.xlabel('x1')  # x軸ラベルの設定
    plt.ylabel('x2')  # y軸ラベルの設定
    plt.legend()  # legendの表示
    plt.show()


# prediction関数 の等高線プロット (fill)
def plot_prediction_torch(model, *args):
    from torch import from_numpy

    # 等高線を描くためのメッシュの生成
    x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
    x1 = x1.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x2 = x2.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x = np.array([x1, x2]).T
    x = from_numpy(x).float()

    #  関数predictionを使って入力xから出力yを計算し、等高線プロットを作成
    y = model(x, *args)
    y = y.detach().numpy()  # PyTroch Tensorからnumpy arrayへの変換
    cs = plt.tricontourf(x[:, 0], x[:, 1], y.flatten(), levels=10)
    plt.colorbar(cs)


中間層が2層、それぞれの層のノード数がそれぞれ3つ、1つのMLPを構成します。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import BCELoss
from torch.optim import SGD
from torch import from_numpy

# データ点の取得
x, t = get_dataset_2()
# numpy -> torch.tensor
x = from_numpy(x).float()
t = from_numpy(t).float()

# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=1.0)

# トレーニング
for i_epoch in range(3000):
    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)


`Linear` は行列計算($y=Wx+b$)をする関数です。
活性化関数(ここでは`Sigmoid`)と組み合わせることで隠れ層を構成します。
`Linear`の詳細は[公式のドキュメント](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)を参照することでわかります。
ドキュメントを見ると、
```python
torch.nn.Linear(
    in_features,
    out_features,
    bias=True,
    dtype=None
)
```
のような引数を持つことがわかります。また、各引数の意味は、
* `in_features`:	size of each input sample.
* `out_features`:	size of each output sample.
* `bias`:	If set to False, the layer will not learn an additive bias. Default: True.

のようになっています。隠れ層の入出力ノードの数、バイアス項の有無が指定できることがわかります。
知らない関数を使うときは、必ずドキュメントを読んで、関数の入出力、引数、デフォルトの値などを確認するようにしましょう。


PyTorch Model (上の例では`model`)は`torchinfo`等の外部パッケージを使用することで、モデルの構成が確認できます。

In [ ]:
from torchinfo import summary
summary(
    model,
    input_size=x.shape,
    col_names=['output_size', 'num_params']
)

このモデルは、１層目の隠れ層の出力が3, 学習可能なパラメータ数が9, 2層目の隠れ層の出力が1, 学習可能なパラメータ数が4 であることがわかります。"Output Shape"の"1100"はバッチサイズです。

層の数を増やしてみましょう。新たな層を重ねることで層の数を増やすことができます。
```python
model = Sequential(
    Linear(in_features=2, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
```

In [ ]:
model = Sequential(
    Linear(in_features=2, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=3),  # ノード数が3の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=3, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

from torchinfo import summary
summary(
    model,
    input_size=x.shape,
    col_names=['output_size', 'num_params']
)


モデルのパラメータの数が増えていることがわかります。

次に、ノードの数を増やしてみましょう。

In [ ]:
model = Sequential(
    Linear(in_features=2, out_features=128),  # ノード数が128の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

from torchinfo import summary
summary(
    model,
    input_size=x.shape,
    col_names=['output_size', 'num_params']
)


パラメータの数が大きく増えたことがわかります。
MLPにおいては、パラメータの数は、ノード数の2乗で増加します。

このモデルを使って学習させてみましょう。

In [ ]:
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(3000):
    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)

これまでは活性化関数としてシグモイド関数(`sigmoid`)を使っていました。昔はsigmoid関数やtanh関数がよく使われていましたが、最近はReLU関数がよく使われます。
$$
  ReLU = \begin{cases}
    x & (x \geq 0) \\
    0 & (x < 0)
  \end{cases}
$$

ReLUが好まれる理由については、別の資料を参照してください。

ReLUを使って学習がどのようになるか確認してみましょう。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import BCELoss
from torch.optim import SGD
from torch import from_numpy

# データ点の取得
x, t = get_dataset_2()
# numpy -> torch.tensor
x = from_numpy(x).float()
t = from_numpy(t).float()

# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(3000):
    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)

深層学習をトレーニングするにあたって、最適化関数(optimizer)も非常に重要な要素です。
確率的勾配降下法(SGD)の他によく使われるアルゴリズムとして adam があります。
adamを使ってみると、どのようになるでしょうか。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import BCELoss
from torch.optim import Adam
from torch import from_numpy

# データ点の取得
x, t = get_dataset_2()
# numpy -> torch.tensor
x = from_numpy(x).float()
t = from_numpy(t).float()

# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = Adam(model.parameters())

# トレーニング
for i_epoch in range(3000):
    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)

## PyTorch モデルの定義方法
PyTorchモデルを定義する方法はいくつかあります。
最も簡単なのが`Sequential`を使った方法で、これまでの例では全てこの方法でモデルを定義してきました。
一方で、少し複雑なモデルを考えると、`Sequential`モデルで対応できなくなってきます。
一例としてResidual Network(ResNet)で使われるskip connectionを考えてみます。
skip connectionは
$$
y = f_2(f_1(x) + x)
$$
のように、入力を２つの経路に分け、片方はMLP、もう片方はそのまま後ろのレイヤーに接続するつなげ方です。
このようなモデルは、途中入出力の分岐があるため、`Sequential`モデルでは実装できません。

PyTorchモデルを定義する方法として、`Module`クラスのサブクラスを作る方法もあります。
`Module`クラスをカスタマイズすることができるので、特殊な学習をさせたいときなど、高度な深層学習モデルを扱うときに使われることもあります。
`forward`という関数の中でモデル内のレイヤーの関係を定義します。

In [ ]:
# Modelクラスを継承して新しいクラスを作成します
from torch.nn import Module
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU

class myModel(Module):
    def __init__(self):
        super().__init__()

        self.linear_1 = Linear(in_features=2, out_features=128)
        self.activation_1 = ReLU()
        self.linear_2 = Linear(in_features=128, out_features=128)
        self.activation_2 = ReLU()
        self.linear_3 = Linear(in_features=128, out_features=128)
        self.activation_3 = ReLU()
        self.linear_4 = Linear(in_features=128, out_features=128)
        self.activation_4 = ReLU()
        self.linear_5 = Linear(in_features=128, out_features=1)
        self.activation_5 = Sigmoid()

    def forward(self, inputs):
        x = self.linear_1(inputs)
        x = self.activation_1(x)
        x = x + self.linear_2(x)  # skip connection
        x = self.activation_2(x)
        x = x + self.linear_3(x)  # skip connection
        x = self.activation_3(x)
        x = x + self.linear_4(x)  # skip connection
        x = self.activation_4(x)
        x = self.linear_5(x)
        x = self.activation_5(x)
        return x

model = myModel()


In [ ]:
from torch.nn import BCELoss
from torch.optim import Adam
from torch import from_numpy

# データ点の取得
x, t = get_dataset_2()
# numpy -> torch.tensor
x = from_numpy(x).float()
t = from_numpy(t).float()

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = Adam(model.parameters())

# トレーニング
for i_epoch in range(3000):
    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)